## NOTE: In general, use Pipeline_Qalf.ipynb and select the domain using the config variable.
This notebook serves primarily as a snapshot of the QalfSpousePipeline working.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'project': 'qalf',
    'domain': 'spouse',
    'supervision': 'majority',
    'disc_model_class': 'logreg',    
    'disc_model_search_space': 1,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'qalf_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///qalf_spouse.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'], config['project'])
pipe = pipeline(session, candidate_class, config)

Overwriting disc_model_search_space=10 to disc_model_search_space=1
Overwriting domain=None to domain=spouse
Overwriting print_freq=1 to print_freq=5
Overwriting disc_model_search_space=10 to disc_model_search_space=1
Overwriting LF_acc_prior_weight_default=1.0 to LF_acc_prior_weight_default=0.5
Overwriting decay=0.95 to decay=0.99
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting babbler_candidate_split=1 to babbler_candidate_split=[0, 1, 2]
Overwriting disc_model_class=lstm to disc_model_class=logreg
Overwriting supervision=generative to supervision=majority
Overwriting project=babble to project=qalf
Using SpouseQalfPipeline object.


In [5]:
# from snorkel.models import Document

# session.query(Document).count()

In [6]:
# for split in [0,1,2]:
#     print(session.query(pipe.candidate_class).filter(
#         pipe.candidate_class.split == split).count())

In [7]:
# %time pipe.parse()

In [8]:
# %time pipe.extract()

In [9]:
# %time pipe.load_gold()

In [10]:
# %time pipe.featurize()

In [11]:
# %time pipe.collect()

In [12]:
%time pipe.label()

Clearing existing...
Running UDF...
[========================================] 100%

Clearing existing...
Running UDF...
[========================================] 100%

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1min 12s, sys: 1.29 s, total: 1min 14s
Wall time: 1min 14s


In [13]:
from snorkel.annotations import load_gold_labels
# L_gold_train = load_gold_labels(session, annotator_name='gold', split=0)
L_gold_dev   = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test  = load_gold_labels(session, annotator_name='gold', split=2)

In [14]:
# L_train.lf_stats(session, labels=L_gold_train)
pipe.L_dev.lf_stats(session, labels=L_gold_dev)
# L_test.lf_stats(session, labels=L_gold_test)

,j,Coverage,Overlaps,Conflicts,TP,FP,FN,TN,Empirical Acc.
Who_is_ENT1_married_to?,0,0.033262,0.030401,0.017525,35,58,0,0,0.376344
Who_is_ENT2_married_to?,1,0.009299,0.008584,0.007153,4,22,0,0,0.153846
Who_got_married_to_ENT1_at_their_wedding?,2,0.013948,0.012160,0.006795,17,22,0,0,0.435897
Who_got_married_to_ENT2_at_their_wedding?,3,0.008941,0.006438,0.003934,9,16,0,0,0.360000
Who_is_ENT1's_husband?,4,0.047210,0.044707,0.023247,58,74,0,0,0.439394
Who_is_ENT2's_husband?,5,0.012518,0.012518,0.008941,7,28,0,0,0.200000
Who_is_ENT1's_wife?,6,0.053648,0.047568,0.025036,57,93,0,0,0.380000
Who_is_ENT2's_wife?,7,0.015379,0.013948,0.009299,7,36,0,0,0.162791
Who_else_is_the_parent_of_a_child_with_ENT1?,8,0.008941,0.008584,0.005722,8,17,0,0,0.320000
Who_else_is_the_parent_of_a_child_with_ENT2?,9,0.001788,0.001788,0.001431,2,3,0,0,0.400000


In [ ]:
%time pipe.supervise()

In [ ]:
%time pipe.classify()